<a href="https://colab.research.google.com/github/PieCic/Challenge-Alura-Google/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install the Google SDK

In [ ]:
!pip install -q -U google-generativeai

Import needed libraries and API Key

In [ ]:
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata
# Import you api key using the secret key
api_key = userdata.get("SECRET_KEY")
genai.configure(api_key=api_key)

List the existing models of Google Generative AI

In [ ]:
for m in genai.list_models():
  if "generateContent" in m.supported_generation_methods:
    print(m.name)

Set configurations and settings of the model

In [47]:
generation_config = {
    "candidate_count": 1,
    "temperature": 0.7,
    "top_k": 40,
    "top_p": 0.95
}

Generate the model with specified configurations and settings

In [48]:
model = genai.GenerativeModel(model_name = "gemini-1.5-pro-latest",
                              generation_config=generation_config)

Generate a text using Gemini and create the embeddings

In [ ]:
# Example of embedding
title = "A Glimpse into the Amazon's Green Symphony"
sample_text = ("Title: A Glimpse into the Amazon's Green Symphony"
"\n"
"""
The Amazon Rainforest, a vibrant tapestry of life, holds the title of the most biodiverse ecosystem on Earth. Within its emerald embrace, a staggering array of plant species thrives, each playing a unique role in the grand orchestra of the jungle.

Towering above the forest floor, giants like the Ceiba pentandra, also known as the Kapok tree, reach for the sunlight with their massive trunks. Their broad leaves form a dense canopy, filtering the sun's rays and creating a dappled world below. Here, epiphytes like orchids and bromeliads, such as the stunning Guzmania lingulata, cling to branches, their roots drawing moisture from the humid air.

Beneath the canopy, a vibrant understory unfolds. Palms like the Euterpe oleracea (Açaí palm) offer their nutrient-rich fruits, while the spiny leaves of the Socratea exorrhiza (Walking Palm) stilt the plant above the flooded forest floor. Lianas, such as the Monstera deliciosa, weave through the vegetation, their thick stems climbing towards the light.

The forest floor, alive with the hum of insects, is a battlefield for sunlight. Here, diverse species of ferns like the Pteris vittata unfurl their delicate fronds, while the vibrant Heliconia flowers attract pollinators with their flamboyant colours. Even the decaying leaves play their part, providing nutrients for fungi and enriching the fertile soil.

The Amazon's plant kingdom is not just a collection of individuals; it's a complex web of interactions. Trees communicate through underground fungal networks, sharing resources and warnings of danger.  Ants cultivate fungus gardens within the hollow chambers of the Cecropia tree, receiving shelter and food in return. This intricate dance of life, from the tallest trees to the smallest seedlings, makes the Amazon Rainforest a true marvel of the natural world.
""")

embeddings = genai.embed_content(model = "models/embedding-001",
                                 content = sample_text,
                                 title = title,
                                 task_type = "RETRIEVAL_DOCUMENT")
print(embeddings)

Generate further text to be used later on

In [ ]:
response = model.generate_content("A Glimpse into the Amazon's Green Symphony")


print(response.text)

Create documents to be placed inside the df dataframe

In [71]:
Doc1 = {
    "Title": "The Orchestra Members",
    "Content": "The Amazon Rainforest is a stage teeming with life, its performers a diverse cast of plants, animals, and people. Towering Kapok trees reach for the sky, while delicate orchids cling to their branches. Jaguars stalk through the undergrowth, and macaws paint the canopy with their vibrant colors. Indigenous communities, with their deep understanding of the forest, act as its guardians, living in harmony with its rhythms. Each member plays a crucial role, from the bees that pollinate the Brazil nut trees to the fungi that decompose fallen leaves, creating a complex and interconnected web of life."
}
Doc2 = {
    "Title": "The Music of the Ecosystem",
    "Content": "The Amazon sings a symphony of sounds, from the chattering of monkeys to the buzzing of insects and the calls of exotic birds. These sounds are not just noise; they are a language of survival, used for communication, finding mates, and warning of danger. The rainforest also dances to the rhythm of life's cycles. Seasons change, rivers rise and fall, and organisms are born, live, and die, creating an ever-evolving environment where each element is connected to the whole."
}
Doc3 = {
    "Title": "Threats to the Symphony",
    "Content": "The Amazon's harmonious melody is facing discordant notes. Deforestation is tearing at the fabric of the forest, destroying habitats and disrupting ecological balance. Climate change brings rising temperatures and erratic weather patterns, further stressing the ecosystem. Pollution from mining and other industries poisons the land and water, while the illegal wildlife trade threatens numerous species with extinction. The very survival of this vibrant symphony is at stake."
}
Doc4 = {
    "Title": "Protecting the Symphony",
    "Content": " Despite the challenges, a chorus of hope rises. Conservation organizations and dedicated individuals are working tirelessly to protect the Amazon. Initiatives to combat deforestation, promote sustainable development, and empower indigenous communities are gaining momentum. Individuals around the world are contributing by supporting sustainable products, reducing their carbon footprint, and raising awareness about the rainforest's importance."
}
Doc5 = {
    "Title": "The Future of the Symphony",
    "Content": "The Amazon, though wounded, remains resilient. Its ability to regenerate and recover offers hope for the future. However, the time to act is now. By joining forces and working together, we can ensure that the Amazon's green symphony continues to resonate for generations to come. Each individual can play a part, becoming a steward of this irreplaceable ecosystem and ensuring that its melody of life continues to inspire and enrich our planet."
}
documents = [Doc1, Doc2, Doc3]

Create a dataframe trhough pandas library

In [ ]:
df = pd.DataFrame(documents)
df

Create the embed_fn function to be used with the elements of df dataframe

In [73]:
model = "models/embedding-001"
def embed_fn(title,text):
  return genai.embed_content(model=model,
                             content=text,
                             title=title,
                             task_type="RETRIEVAL_DOCUMENT")["embedding"]

df["embeddings"] = df.apply(lambda row: embed_fn(row["Title"], row["Content"]), axis=1)
df

Create the function for which the AI generate and search content through your embeddings

In [75]:
def generate_and_search_consult(consult,base,model):
  embedding_consult = genai.embed_content(model=model,
                                          content=consult,
                                          task_type="RETRIEVAL_DOCUMENT")["embedding"]
  scalar_product = np.dot(np.stack(df["embeddings"]), embedding_consult)

  index = np.argmax(scalar_product)
  return df.loc[index]["Content"]

Ask the AI about df elements

In [ ]:
consult = "What is the Future of the Symphony of the Amazon?"

paragraph = generate_and_search_consult(consult,df,model)
print(paragraph)

Ask the AI to rewrite the text for a first grade kid

In [ ]:
prompt = f"Rewrite this text: {paragraph}, to be easy to understand for a child of first grade"

model_2 = genai.GenerativeModel("gemini-1.0-pro")
response = model_2.generate_content(prompt)
print(response.text)